In [26]:
from langchain_google_vertexai import VertexAI, VertexAIEmbeddings, ChatVertexAI
import vertexai
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_vertexai import ChatVertexAI
from pathlib import Path
from langchain_core.output_parsers import StrOutputParser

def context_gen(file_name):
    Folder = "VertexLang"
    here = Path(locals().get('__file__', Folder)).resolve()
    parameter = (here / file_name).read_text()
    return parameter

def remove_code_fences(text):
    lines = text.split('-')
    lines = [line for line in lines if not line.strip().startswith(' ')]
    return '\n'.join(lines)


def test_vertexai(string):
    import textwrap
    prompt = ChatPromptTemplate.from_messages([("system", system), ("user", input_examples), ("assistant", schema), ("human", string)])
    chat = ChatVertexAI()
    chain = prompt | chat | output_parser
    result = chain.invoke({})
    print(remove_code_fences(result))
    print()
    return



vertexai.init(project = "totemic-veld-412608")
llm = ChatVertexAI(model_name = "gemini-1.0-pro", 
                temperature = "0.5", 
                max_output_tokens = 1000)

output_parser = StrOutputParser()


system = context_gen("systemmsg.txt")
input_examples = context_gen("inputex.txt")
schema = context_gen("dataBaseSchema.txt")
human = ("select all boltholes and then select their corresponding edges and then select all flanges")
test_vertexai(human)

 
 filter: select * from faces where type like "Cylindrical"

 expand: faceedges

 filter: select * from faces where type like "Planar"

